In [1]:
import pandas as pd
import dask as dd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt
import statsmodels.formula.api as smf
import statistics

from sklearn.linear_model import LogisticRegression

import statsmodels.api as sm
from statsmodels.genmod.generalized_linear_model import GLM
from statsmodels.genmod import families
from statsmodels.stats.outliers_influence import variance_inflation_factor

from scipy import stats

/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [17]:
def basePrep(df):
	df['rndf'] = df['rndf'].map({"Yes": 1, "No": 0})
	df['pdynamic'] = df['pdynamic'].map({'new_entrant': 1, 'incumbent': 0})	
	df["SNATION_PARTAL".lower()] = df["SNATION_PARTAL".lower()].map({"Y": 1, "N": 0})
	
	cols = ['rndf', 'ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count', 'pdynamic', "SNATION_PARTAL".lower()]
	df = df[cols].dropna()
	
	return df

## Try all potential variables

In [18]:
df = pd.read_csv('./data/JV_data_dist.csv')

In [19]:
df = basePrep(df)

In [21]:
cols = ['rndf', 'ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count', "SNATION_PARTAL".lower()]
X = (df[cols]).astype(float)
y = df['pdynamic'].astype(float)
X_constant = sm.add_constant(X, prepend=False)

In [22]:
logit_model = GLM(y, X_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  729
Model:                            GLM   Df Residuals:                      721
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -436.89
Date:                Wed, 15 Jun 2022   Deviance:                       873.77
Time:                        19:30:30   Pearson chi2:                     769.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1102
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf               0.4712      0.303      1.

## Filter only R&D alliances

In [23]:
df2 = pd.read_csv('./data/JV_data_dist.csv')

df2 = df2[df2['rndf'] == "Yes"]
df2 = basePrep(df2)

cols2 = ['ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count', "SNATION_PARTAL".lower()]
X2 = (df2[cols2]).astype(float)
y2 = df2['pdynamic'].astype(float)
X2_constant = sm.add_constant(X2, prepend=False)

logit_model = GLM(y2, X2_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                   66
Model:                            GLM   Df Residuals:                       59
Model Family:                Binomial   Df Model:                            6
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -33.273
Date:                Wed, 15 Jun 2022   Deviance:                       66.547
Time:                        19:31:12   Pearson chi2:                     62.2
No. Iterations:                     5   Pseudo R-squ. (CS):             0.1749
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
ddist_bin         -0.1216      1.217     -0.

## Filter for alliance SIC

In [24]:
df3 = pd.read_csv("./data/JV_data_dist.csv")

#Select alliances that at least part of their goal was manufacturing of cars
df3 = df3[df3['sic'].str.contains('3711') == True]
df3 = basePrep(df3)

cols3 = ['rndf', 'ddist_bin','ddist_int', 'ddist_abs', 'avg_emp_pp', 'public_count', "SNATION_PARTAL".lower()]
X3 = (df3[cols3]).astype(float)
y3 = df3['pdynamic'].astype(float)
X3_constant = sm.add_constant(X3, prepend=False)

logit_model = GLM(y3, X3_constant, family=families.Binomial())
logit_results = logit_model.fit()
print(logit_results.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               pdynamic   No. Observations:                  311
Model:                            GLM   Df Residuals:                      303
Model Family:                Binomial   Df Model:                            7
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -187.00
Date:                Wed, 15 Jun 2022   Deviance:                       373.99
Time:                        19:31:48   Pearson chi2:                     330.
No. Iterations:                     4   Pseudo R-squ. (CS):             0.1595
Covariance Type:            nonrobust                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
rndf              -0.5976      0.722     -0.